In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plot


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#initialization of variables
alpha=.01
units=128
real=784
z=100
sfactor=.1
batches=100


In [4]:
#input data function
def input_para(real,z):
    inp_real=tf.placeholder(tf.float32,[None,real])
    z_real=tf.placeholder(tf.float32,[None,z])
    return inp_real,z_real
#using matplotplib to display images generated by generator while training
def display(sample):
    image = sample.reshape([280,280])
    plot.axis('off')
    plot.imshow(image, cmap=matplotlib.cm.binary)
    plot.show()

In [5]:
#discriminator function with 1 hidden layer
def generator(z,units,g_out,reuse=False):
    with tf.variable_scope('generator',reuse=reuse):
        hidden=tf.layers.dense(z,units,activation=None)
        hidden=tf.maximum(hidden,alpha*hidden)
        logits=tf.nn.tanh(tf.layers.dense(hidden,g_out,activation=None))
        return logits


#generator function with 1 hidden layer
def discriminator(real,units,out,reuse=False):
    with tf.variable_scope('discriminator',reuse=reuse):
        hidden=tf.layers.dense(real,units,activation=None)
        hidden=tf.maximum(hidden,alpha*hidden)
        logits=tf.layers.dense(hidden,out,activation=None)
        out=tf.nn.sigmoid(logits)
        return out,logits

In [6]:
tf.reset_default_graph()
#calling the functions and making the model
inp_real,z_real=input_para(real,z) 
g_out=generator(z_real,units,784,reuse=False)
d_out_real,d_logits_real=discriminator(inp_real,units,1,reuse=False)
d_out_fake,d_logits_fake=discriminator(g_out,units,1,reuse=True)

In [7]:

#loss finding
d_labels_real = tf.ones_like(d_logits_real) * (1 - sfactor)
d_labels_fake = tf.zeros_like(d_logits_fake)

d_loss_real = tf.nn.sigmoid_cross_entropy_with_logits(labels=d_labels_real, logits=d_logits_real)
d_loss_fake = tf.nn.sigmoid_cross_entropy_with_logits(labels=d_labels_fake, logits=d_logits_fake)

d_loss = tf.reduce_mean(d_loss_real + d_loss_fake)

g_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        labels=tf.ones_like(d_logits_fake), 
        logits=d_logits_fake))


In [8]:

#optimization
t = tf.trainable_variables()
g_vars = [var for var in t if var.name.startswith('generator')]
d_vars = [var for var in t if var.name.startswith('discriminator')]
d_train_opt = tf.train.AdamOptimizer(.001).minimize(d_loss, var_list=d_vars)
g_train_opt = tf.train.AdamOptimizer(.001).minimize(g_loss, var_list=g_vars)


In [ ]:
epochs=10000
batch_size=100
z_size=100
samples = []
losses = []
saver = tf.train.Saver(var_list = g_vars)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for ii in range(mnist.train.num_examples//batch_size):
            batch = mnist.train.next_batch(batch_size)
            
            # Get images, reshape and rescale to pass to D
            batch_images = batch[0].reshape((batch_size, 784))
            batch_images = batch_images*2 - 1
            
            # Sample random noise for G
            batch_z = np.random.uniform(-1, 1, size=(batch_size, z_size))
            
            # Run optimizers
            _ = sess.run(d_train_opt, feed_dict={inp_real: batch_images, z_real: batch_z})
            _ = sess.run(g_train_opt, feed_dict={z_real: batch_z})
        
        # At the end of each epoch, get the losses and print them out
        train_loss_d = sess.run(d_loss, {z_real: batch_z, inp_real: batch_images})
        train_loss_g = g_loss.eval({z_real: batch_z})
        if e%100==0:    
            sample=sess.run(g_out,feed_dict={z_real: batch_z})
            print("Epoch {}/{}...".format(e+1, epochs),
              "Discriminator Loss: {:.4f}...".format(train_loss_d),
              "Generator Loss: {:.4f}".format(train_loss_g))    
              # Save losses to view after training
            losses.append((train_loss_d, train_loss_g))  

('Epoch 1/10000...', 'Discriminator Loss: 0.3541...', 'Generator Loss: 4.0235')
('Epoch 101/10000...', 'Discriminator Loss: 0.9313...', 'Generator Loss: 1.8187')
('Epoch 201/10000...', 'Discriminator Loss: 1.0625...', 'Generator Loss: 1.4532')
('Epoch 301/10000...', 'Discriminator Loss: 0.9270...', 'Generator Loss: 1.5948')
('Epoch 401/10000...', 'Discriminator Loss: 1.1227...', 'Generator Loss: 1.5980')
('Epoch 501/10000...', 'Discriminator Loss: 0.9848...', 'Generator Loss: 2.2758')
('Epoch 601/10000...', 'Discriminator Loss: 0.9784...', 'Generator Loss: 1.9068')
('Epoch 701/10000...', 'Discriminator Loss: 0.9388...', 'Generator Loss: 1.9230')
('Epoch 801/10000...', 'Discriminator Loss: 0.8380...', 'Generator Loss: 1.9166')
('Epoch 901/10000...', 'Discriminator Loss: 0.9508...', 'Generator Loss: 1.8653')
('Epoch 1001/10000...', 'Discriminator Loss: 0.9668...', 'Generator Loss: 1.7162')
('Epoch 1101/10000...', 'Discriminator Loss: 0.9491...', 'Generator Loss: 1.7107')
('Epoch 1201/100